In [1]:
using JLD2
using Statistics
using Revise
using Plots
using Laplacians
using Printf

In [2]:
function sd(d,x)
    #scale = exp10(ceil(log10(abs(x)))-d)
    #return round(x/scale)*scale
    return round(x, sigdigits=d)
end        
ff(s) = (@sprintf "%.4g" sd(3,s))
ffK(s) = (@sprintf "%.5g" sd(3,s)/1e3)
ffM(s) = (@sprintf "%.3g" sd(3,s)/1e6)
ffmicro(s) = (@sprintf "%.4g" sd(3,s)*1e6)
ffe8(s) = (@sprintf "%.4g" sd(3,s)*1e8)

ffe8 (generic function with 1 method)

In [3]:
@load "cmg2_test_small.jl.jld2"
dic_cmg2 = copy(dic)

Dict{Any,Any} with 9 entries:
  "cmg_solve" => [3.02144, 0.315614, 0.312859, 0.148132, 0.261944, 0.290326, 0.…
  "names"     => ["cmg"]
  "cmg_build" => [0.847754, 0.400594, 0.333275, 0.087946, 0.239077, 0.349053, 0…
  "ne"        => [4996000, 327680, 197608, 3240, 87025, 197608, 4617, 7744, 313…
  "testName"  => ["McRae/ecology1", "MaxPlanck/shallow_water2", "GHS_psdef/tors…
  "cmg_tot"   => [3.86919, 0.716208, 0.646134, 0.236078, 0.501021, 0.639379, 0.…
  "nv"        => [1000000, 81920, 40000, 3240, 9801, 40000, 729, 900, 3134, 356…
  "cmg_its"   => [28.0, 15.0, 19.0, 2.0, 22.0, 19.0, 19.0, 22.0, 2.0, 2.0  …  2…
  "cmg_err"   => [6.12387e-9, 5.72918e-9, 9.70493e-9, 1.33409e-16, 9.4121e-9, 9…

In [4]:
uni_chimera_indices = findall(entry -> contains(entry, "uni_chimera(10000,"), dic_cmg2["testName"])
wted_chimera_indices = findall(entry -> contains(entry, "wted_chimera(10000,"), dic_cmg2["testName"])
uni_bndry_chimera_indices = findall(entry -> contains(entry, "uni_bndry_chimera(10000,"), dic_cmg2["testName"])
wted_bndry_chimera_indices = findall(entry -> contains(entry, "wted_bndry_chimera(10000,"), dic_cmg2["testName"])
suitesparse_indices = setdiff(1:length(dic_cmg2["testName"]), vcat(uni_chimera_indices, wted_chimera_indices, uni_bndry_chimera_indices, wted_bndry_chimera_indices));


In [5]:
problem_idx = [suitesparse_indices, uni_chimera_indices, wted_chimera_indices, uni_bndry_chimera_indices, wted_bndry_chimera_indices]
problem_names = ["suitesparse", "uni_chimera", "wted_chimera", "uni_bndry_chimera", "wted_bndry_chimera"]

5-element Array{String,1}:
 "suitesparse"
 "uni_chimera"
 "wted_chimera"
 "uni_bndry_chimera"
 "wted_bndry_chimera"

In [6]:
ratios = []
errors = []
for indices in problem_idx
    @show length(indices)
    ratio = dic_cmg2["cmg_tot"][indices] ./ dic_cmg2["ne"][indices]
    push!(ratios, ratio)
    error = dic_cmg2["cmg_err"][indices]
    push!(errors, error)
end


length(indices) = 28
length(indices) = 5
length(indices) = 0
length(indices) = 0
length(indices) = 0


In [7]:
for (name, ratio, error) in zip(problem_names, ratios, errors)
    mean_val = mean(ratio)
    quantile_val = quantile(ratio, 0.75)
    max_val = maximum(ratio)
    
    @printf("%-20s - Ratio - Mean: %10s | 0.75 Quantile: %10s | Max: %10s\n", name, ffmicro(mean_val), ffmicro(quantile_val), ffmicro(max_val))
    
    mean_error = mean(error)
    quantile_error = quantile(error, 0.75)
    max_error = maximum(error)
    
    @printf("\u001b[31m%-20s - Error - Mean: %10s | 0.75 Quantile: %10s | Max: %10s\n\u001b[0m", name, ffe8(mean_error), ffe8(quantile_error), ffe8(max_error))
end

suitesparse          - Ratio - Mean:       45.2 | 0.75 Quantile:       63.3 | Max:        207
suitesparse          - Error - Mean:      0.419 | 0.75 Quantile:      0.875 | Max:      0.985
uni_chimera          - Ratio - Mean:       8.02 | 0.75 Quantile:       10.8 | Max:         14
uni_chimera          - Error - Mean:      0.643 | 0.75 Quantile:      0.808 | Max:      0.864


LoadError: [91mArgumentError: empty data vector[39m

In [8]:
failure_idx = findall(entry -> entry == 1.0 || entry == Inf, dic_cmg2["cmg_err"])
failure_names = dic_cmg2["testName"][failure_idx]


String[]